<p align="center">
  <img src="https://wallpapersok.com/images/high/juice-wrld-desktop-digital-art-7sdyvhtfy9h9mths.webp" width="600">
</p>

# **Analyzing Song Lyrics - Juice WRLD**  

a-sofia-p

In [4]:
#!pip install lyricsgenius
#!pip install bertopic[visualization]
#!pip install python-dotenv

In [45]:
import pandas as pd 
import lyricsgenius 
import requests
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from bertopic import BERTopic
from umap import UMAP
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import altair as alt
import os
from dotenv import load_dotenv

## Using Genius API to get 100 Juice WRLD songs

In [42]:
# using dotenv to hide Genius API User Key
load_dotenv()
api_key = os.getenv("GENIUS_API_KEY")

NameError: name 'load_dotenv' is not defined

In [38]:
genius = lyricsgenius.Genius(api_key)

In [40]:
# fetching 100 songs by Juice WRLD
artist = genius.search_artist("Juice WRLD", max_songs=100, sort="popularity")

artist.save_lyrics("juice_wrld_lyrics.json")
print(f"Lyrics for {artist.name} have been saved!")

Searching for songs by Juice WRLD...

Song 1: "Lucid Dreams"
Song 2: "All Girls Are the Same"
Song 3: "Wishing Well"
Song 4: "Robbery"
Song 5: "Legends"
Song 6: "Bandit"
Song 7: "Righteous"
Song 8: "Lean wit Me"
Song 9: "Wasted"
Song 10: "Smile"
Song 11: "Come & Go"
Song 12: "Empty"
Song 13: "Life’s a Mess"
Song 14: "Armed and Dangerous"
Song 15: "Blood On My Jeans"
Song 16: "Hate the Other Side"
Song 17: "Cigarettes"
Song 18: "Let Me Know (I Wonder Why Freestyle)"
Song 19: "​moonlight"
Song 20: "Conversations"
Song 21: "Sometimes"
Song 22: "Bad Boy"
Song 23: "Fast"
Song 24: "Burn"
Song 25: "​autograph (on my line)"
Song 26: "Rental"
Song 27: "Rich and Blind"
Song 28: "Stay High"
Song 29: "Black & White"
Song 30: "Already Dead"
Song 31: "Flaws and Sins"
Song 32: "Hear Me Calling"
Song 33: "Man of the Year"
Song 34: "RUN"
Song 35: "Tell Me U Luv Me"
Song 36: "Hide"
Song 37: "Scared Of Love"
Song 38: "Girl Of My Dreams"
Song 39: "Candles"
Song 40: "I’ll Be Fine"
Song 41: "I’m Still"
Song

### Loading JSON file with Juice WRLD song lyrics fetched through Genius API

In [13]:
with open("juice_wrld_lyrics.json", "r") as file:
    data = json.load(file)

### Exploring the downloaded JSON file

In [20]:
print(data.keys())

dict_keys(['alternate_names', 'api_path', 'description', 'facebook_name', 'header_image_url', 'id', 'image_url', 'instagram_name', 'is_meme_verified', 'is_verified', 'name', 'translation_artist', 'twitter_name', 'url', 'current_user_metadata', 'followers_count', 'iq', 'description_annotation', 'user', 'songs'])


In [ ]:
# accessing the 'songs' element from the JSON file
songs = data.get("songs", [])

In [18]:
print(type(data['songs'])) 
print(data['songs'][0].keys())

dict_keys(['annotation_count', 'api_path', 'artist_names', 'full_title', 'header_image_thumbnail_url', 'header_image_url', 'id', 'lyrics_owner_id', 'lyrics_state', 'path', 'primary_artist_names', 'pyongs_count', 'relationships_index_url', 'release_date_components', 'release_date_for_display', 'release_date_with_abbreviated_month_for_display', 'song_art_image_thumbnail_url', 'song_art_image_url', 'stats', 'title', 'title_with_featured', 'url', 'featured_artists', 'primary_artist', 'primary_artists', 'apple_music_id', 'apple_music_player_url', 'description', 'embed_content', 'featured_video', 'language', 'recording_location', 'release_date', 'current_user_metadata', 'song_art_primary_color', 'song_art_secondary_color', 'song_art_text_color', 'album', 'custom_performances', 'description_annotation', 'lyrics_marked_complete_by', 'lyrics_marked_staff_approved_by', 'media', 'producer_artists', 'song_relationships', 'translation_songs', 'verified_annotations_by', 'verified_contributors', 'ver

In [16]:
#print(data['songs'][1]) 

<class 'list'>
{'annotation_count': 5, 'api_path': '/songs/3389874', 'artist_names': 'Juice WRLD', 'full_title': 'All Girls Are the Same by\xa0Juice\xa0WRLD', 'header_image_thumbnail_url': 'https://images.genius.com/996f368e1cf124520ac2c844c8dd024b.300x300x1.png', 'header_image_url': 'https://images.genius.com/996f368e1cf124520ac2c844c8dd024b.939x939x1.png', 'id': 3389874, 'lyrics_owner_id': 3645670, 'lyrics_state': 'complete', 'path': '/Juice-wrld-all-girls-are-the-same-lyrics', 'primary_artist_names': 'Juice WRLD', 'pyongs_count': 246, 'relationships_index_url': 'https://genius.com/Juice-wrld-all-girls-are-the-same-sample', 'release_date_components': {'year': 2017, 'month': 12, 'day': 22}, 'release_date_for_display': 'December 22, 2017', 'release_date_with_abbreviated_month_for_display': 'Dec. 22, 2017', 'song_art_image_thumbnail_url': 'https://images.genius.com/996f368e1cf124520ac2c844c8dd024b.300x300x1.png', 'song_art_image_url': 'https://images.genius.com/996f368e1cf124520ac2c844c

### Function to Remove Section Headers  

The function takes song lyrics, such as "[Chorus]\nHello, it's me\n[Verse 1]\nI was wondering...", and strips away [Chorus], [Verse_1], for example.

In [300]:
# Function to remove section headers
def remove_section_headers(lyrics):
    return re.sub(r'\[.*?\]', '', lyrics).strip()

### Function to Subset by Song Title, Lyrics and Release Date

In [50]:
# Function to filter out songs based on title and lyrics
def process_songs(songs, excluded_terms):
    processed_songs = []

    excluded_terms = ['(Remix)', '(remix)', '(Live)', '(live)', '(sample)', '(Sample)', '(Demo)', '(demo)']
    
    for song in songs:
        title = song["title"]
        lyrics = song["lyrics"]
        date = song['release_date_components']
        
        if any(term in title for term in excluded_terms):
            continue

        cleaned_lyrics = remove_section_headers(lyrics)

        if title.lower() == "track list" or len(cleaned_lyrics) == 0:
            continue  
            
        processed_songs.append({"title": title, "lyrics": cleaned_lyrics, 'release_date_components': date})
    
    return processed_songs

In [486]:
processed_songs = process_songs(songs, excluded_terms)

### Save processed songs to JSON file

In [308]:
with open("cleaned_songs.json", "w") as outfile:
    json.dump({"songs": processed_songs}, outfile, indent=4)

### Load new JSON file of subset 

In [311]:
with open("cleaned_songs.json", "r") as filec:
    data_clean = json.load(filec)

print(type(data_clean["songs"]))  
print(data_clean["songs"][1]) 

<class 'list'>
{'title': 'All Girls Are the Same', 'lyrics': "292 ContributorsTranslationsTürkçeEspañolРусскийPortuguêsItalianoDeutschFrançaisNederlandsLatviešuУкраїнськаAll Girls Are the Same Lyrics“All Girls Are the Same” flips the rap stereotype of an artist conquering a plethora of women with a Juice WRLD signature – a slow moving track about sorrow and heartbreak.\n\nCole Bennett and Lyrical Lemonade… Read More\xa0\nMhm, they're rotting my brain, love\nThese hoes are the same\n\n\nI admit it, another ho got me finished\nBroke my heart, oh, no, you didn't\nFuck sippin', I'ma down a whole bottle\nHard liquor, hard truth, can't swallow\nNeed a bartender, put me out my sorrow\nWake up the next day in the Monte Carlo\nWith a new woman, tell me she from Colorado\nAnd she love women, she'll be gone by tomorrow\nWho am I kiddin'?\nAll this jealousy and agony that I sit in\nI'm a jealous boy, really feel like John Lennon\nI just want real love, guess it's been a minute\nPissed off from the

### Create DataFrame

In [314]:
juice_df = pd.DataFrame(data_clean)
juice_df = pd.DataFrame(juice_df['songs'].tolist())
juice_df

,songs
0,"{'title': 'Lucid Dreams', 'lyrics': '540 Contr..."
1,"{'title': 'All Girls Are the Same', 'lyrics': ..."
2,"{'title': 'Wishing Well', 'lyrics': '207 Contr..."
3,"{'title': 'Robbery', 'lyrics': '284 Contributo..."
4,"{'title': 'Legends', 'lyrics': '383 Contributo..."
...,...
94,"{'title': 'Ronald', 'lyrics': '124 Contributor..."
95,"{'title': 'Arctic Tundra [Original]', 'lyrics'..."
96,"{'title': 'Adore You', 'lyrics': '87 Contribut..."
97,"{'title': 'Confide', 'lyrics': '77 Contributor..."


In [318]:
print(extracted_data['lyrics'][18])

Yeah, mhm-mm
Mhm
Enviyon on the mix
Yeah


Girl, you know I'm far from perfect, I won't lead you on
But tonight is feelin' perfect, this your favorite song
I know that mood right
I know that mood right, oh
Dance in the moonlight
Dance in the moonlight, oh
Dance in the moonlight, oh
Dance in the moonlight, oh
Dance in the moonlight
Dance in the moonlight, oh
Dance in the moonlight, oh
Dance in the moonlight, oh, oh, oh, oh
Dance in the moonlight, oh


Don't know when I met you, but I met you
Don't know why I love you, but I love you, oh
It's something 'bout that dress, the way it fits you
I don't even wanna tell you to take it off, but baby, take it off
We in this party on molly, no time for monologue
She say I'm Kevin Hart funny, so she gon' ride along
She say she came in with endurance, so she gon' do me all night long (Take it off)
We could dance in the moonlight, oh
She got a man, I told him, "Come to the moonlight brawl"
Don't stand a chance, don't stand a chance in the moonlight, 

# Preprocessing lyrics

In [321]:
'''Function to remove the unwanted section before the actual lyrics by matching any number at the beginning followed by any characters until "More"'''
def remove_pre_lyrics_section(lyrics):
    pattern = r'^\d+.*?More\b'
    cleaned_lyrics = re.sub(pattern, '', lyrics, flags=re.DOTALL).strip()
    return cleaned_lyrics

In [324]:
juice_df['lyrics'] = juice_df['lyrics'].apply(remove_pre_lyrics_section) 
juice_df

,title,lyrics,release_date_components
0,Lucid Dreams,"Enviyon on the mix\nNo, no, no, no\nNo-no, no,...","{'year': 2017, 'month': 6, 'day': 15}"
1,All Girls Are the Same,"Mhm, they're rotting my brain, love\nThese hoe...","{'year': 2017, 'month': 12, 'day': 22}"
2,Wishing Well,"Mm-mm, mm-mm-mm-mm, mm-mm-mm, uh\nI can't brea...","{'year': 2020, 'month': 7, 'day': 10}"
3,Robbery,"Man, what?\nThis shit funny, one second\nOof, ...","{'year': 2019, 'month': 2, 'day': 13}"
4,Legends,"I usually don't speak on shit like this, but\n...","{'year': 2018, 'month': 6, 'day': 19}"
...,...,...,...
94,Ronald,"Uh, yeah, huh\n\n\nWhat's that belt? Hermès, u...","{'year': 2019, 'month': 3, 'day': None}"
95,Arctic Tundra [Original],"808 Mafia\nHaha, Max Lord, preach it, haha\nGe...",None
96,Adore You,"Let me talk to y'all for a second, though\nLet...","{'year': 2024, 'month': 11, 'day': 29}"
97,Confide,"Uh, I need to confide\nI need to get right\nIt...",None


In [326]:
type(juice_df['release_date_components'])

pandas.core.series.Series

In [494]:
juice_df['year'] = juice_df['release_date_components'].apply(lambda x: x.get('year') if isinstance(x, dict) else None)
#juice_df

In [452]:
nltk.download('stopwords')
nltk.download('punkt')

''' Function that removes section headers, punctiations, numbers and stopwords'''
def preprocess_lyrics(lyrics):
    lyrics = re.sub(r"\[.*?\]", "", lyrics)
    lyrics = re.sub(r"[^\w\s]", "", lyrics)
    lyrics = re.sub(r"\d+", "", lyrics)
    
    tokens = word_tokenize(lyrics.lower())
    filtered_tokens = [word for word in tokens if word not in stopwords.words("english")]
  
    return " ".join(filtered_tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sofia\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sofia\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [496]:
juice_df['cleaned_lyrics'] = juice_df['lyrics'].apply(preprocess_lyrics)
juice_df

,title,lyrics,release_date_components,year,topics,topic,cleaned_lyrics
0,Lucid Dreams,"Enviyon on the mix\nNo, no, no, no\nNo-no, no,...","{'year': 2017, 'month': 6, 'day': 15}",2017.0,2,1,enviyon mix nono still see shadows room cant t...
1,All Girls Are the Same,"Mhm, they're rotting my brain, love\nThese hoe...","{'year': 2017, 'month': 12, 'day': 22}",2017.0,-1,1,mhm theyre rotting brain love hoes admit anoth...
2,Wishing Well,"Mm-mm, mm-mm-mm-mm, mm-mm-mm, uh\nI can't brea...","{'year': 2020, 'month': 7, 'day': 10}",2020.0,0,-1,mmmm mmmmmmmm mmmmmm uh cant breathe chopsquad...
3,Robbery,"Man, what?\nThis shit funny, one second\nOof, ...","{'year': 2019, 'month': 2, 'day': 13}",2019.0,-1,-1,man shit funny one second oof hennessy strong ...
4,Legends,"I usually don't speak on shit like this, but\n...","{'year': 2018, 'month': 6, 'day': 19}",2018.0,0,1,usually dont speak shit like aint nobody else ...
...,...,...,...,...,...,...,...
79,Ronald,"Uh, yeah, huh\n\n\nWhat's that belt? Hermès, u...","{'year': 2019, 'month': 3, 'day': None}",2019.0,-1,0,uh yeah huh whats belt hermès uh uhhuh whats n...
80,Arctic Tundra [Original],"808 Mafia\nHaha, Max Lord, preach it, haha\nGe...",None,NaN,-1,0,mafia haha max lord preach haha gezin fuck blu...
81,Adore You,"Let me talk to y'all for a second, though\nLet...","{'year': 2024, 'month': 11, 'day': 29}",2024.0,-1,-1,let talk yall second though let get lights let...
82,Confide,"Uh, I need to confide\nI need to get right\nIt...",None,NaN,2,1,uh need confide need get right long ride uh dr...


In [456]:
# Drop the rows with the specified indices
juice_df = juice_df.drop([18, 47, 52, 60, 68])
juice_df = juice_df.reset_index(drop=True)
juice_df

,title,lyrics,release_date_components,year,topics,topic,cleaned_lyrics
0,Lucid Dreams,"Enviyon on the mix\nNo, no, no, no\nNo-no, no,...","{'year': 2017, 'month': 6, 'day': 15}",2017.0,2,2,enviyon mix nono still see shadows room cant t...
1,All Girls Are the Same,"Mhm, they're rotting my brain, love\nThese hoe...","{'year': 2017, 'month': 12, 'day': 22}",2017.0,-1,-1,mhm theyre rotting brain love hoes admit anoth...
2,Wishing Well,"Mm-mm, mm-mm-mm-mm, mm-mm-mm, uh\nI can't brea...","{'year': 2020, 'month': 7, 'day': 10}",2020.0,0,1,mmmm mmmmmmmm mmmmmm uh cant breathe chopsquad...
3,Robbery,"Man, what?\nThis shit funny, one second\nOof, ...","{'year': 2019, 'month': 2, 'day': 13}",2019.0,-1,-1,man shit funny one second oof hennessy strong ...
4,Legends,"I usually don't speak on shit like this, but\n...","{'year': 2018, 'month': 6, 'day': 19}",2018.0,0,1,usually dont speak shit like aint nobody else ...
...,...,...,...,...,...,...,...
79,Ronald,"Uh, yeah, huh\n\n\nWhat's that belt? Hermès, u...","{'year': 2019, 'month': 3, 'day': None}",2019.0,-1,-1,uh yeah huh whats belt hermès uh uhhuh whats n...
80,Arctic Tundra [Original],"808 Mafia\nHaha, Max Lord, preach it, haha\nGe...",None,NaN,-1,-1,mafia haha max lord preach haha gezin fuck blu...
81,Adore You,"Let me talk to y'all for a second, though\nLet...","{'year': 2024, 'month': 11, 'day': 29}",2024.0,-1,-1,let talk yall second though let get lights let...
82,Confide,"Uh, I need to confide\nI need to get right\nIt...",None,NaN,2,-1,uh need confide need get right long ride uh dr...


In [500]:
nan_rows = juice_df[juice_df['year'].isna()]
#nan_rows

In [498]:
#nan_rows['title'].values

In [462]:
# Group the DataFrame by the 'year' column and aggregate the song titles (you can change how you want to aggregate)
songs_by_year = juice_df.groupby('year')['title'].apply(list)
songs_by_year

year
2015.0                                            [Forever]
2017.0    [Lucid Dreams, All Girls Are the Same, Let Me ...
2018.0    [Legends, Lean wit Me, Wasted, Armed and Dange...
2019.0    [Robbery, Bandit, Empty, Fast, Flaws and Sins,...
2020.0    [Wishing Well, Righteous, Smile, Come & Go, Li...
2021.0    [Bad Boy, Burn, Already Dead, Girl Of My Dream...
2022.0                                         [Cigarettes]
2023.0                                   [Lace It, No Good]
2024.0     [Party By Myself, KTM Drip, Floor It, Adore You]
2025.0                                             [Rental]
Name: title, dtype: object

# Bag of Words

In [509]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(juice_df['cleaned_lyrics'])
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
#bow_df

In [511]:
word_frequencies = bow_df.sum().sort_values(ascending=False)

frequency_df = word_frequencies.reset_index()
frequency_df.columns = ['word', 'frequency']
frequency_df

,word,frequency
0,yeah,854
1,im,793
2,like,445
3,uh,397
4,dont,385
...,...,...
3640,lames,1
3641,lame,1
3642,lamborghin,1
3643,lake,1


In [523]:
vector_bigram = CountVectorizer(ngram_range=(3,3)) #tried unigram and bigrams but the output is not very useful
X = vector_bigram.fit_transform(juice_df['cleaned_lyrics'])
bow_bigram_df = pd.DataFrame(X.toarray(), columns=vector_bigram.get_feature_names_out())
bigram_frequencies = bow_bigram_df.sum().sort_values(ascending=False)

bigram_frequency_df = bigram_frequencies.reset_index()
bigram_frequency_df.columns = ['word', 'frequency']
#bigram_frequency_df

In [533]:
filtered_frequencies = bigram_frequency_df[bigram_frequency_df['frequency'] > 10]
filtered_frequencies

,word,frequency
0,yeah yeah yeah,108
1,blade blade blade,48
2,know let know,37
3,let know let,37
4,oh oh oh,31
5,way many huh,26
6,many huh yeah,26
7,tell love tell,25
8,dont wan na,25
9,woah woah woah,24


In [540]:
juice_df['total_words'] = juice_df['cleaned_lyrics'].apply(lambda x: len(x.split()))
juice_df['unique_words'] = juice_df['cleaned_lyrics'].apply(lambda x: len(set(x.split())))
juice_df[['title', 'total_words', 'unique_words', 'year']]

,title,total_words,unique_words,year
0,Lucid Dreams,278,87,2017.0
1,All Girls Are the Same,174,107,2017.0
2,Wishing Well,232,97,2020.0
3,Robbery,237,98,2019.0
4,Legends,191,87,2018.0
...,...,...,...,...
79,Ronald,306,131,2019.0
80,Arctic Tundra [Original],248,117,NaN
81,Adore You,161,97,2024.0
82,Confide,197,91,NaN


# Visualization of total words vs. unique words  

- word frequency is calculated over each song
- visualization is per year

In [550]:
chart = alt.Chart(juice_df).mark_bar().encode(
    x=alt.X('year:O', title='Year'),
    y=alt.Y('value:Q', title='Word Count'),
    color=alt.Color('variable:N', title='Word Type', scale=alt.Scale(domain=['total_words', 'unique_words'],
                                                                      range=['#1f77b4', '#ff7f0e'])),
    column=alt.Column('variable:N', title=None),
    tooltip=['year:O', 'value:Q']
).transform_fold(
    ['total_words', 'unique_words'],
    as_=['variable', 'value']
).properties(
    width=300,
    height=400,
    title='Total Words and Unique Words Per Year'
)

chart

alt.Chart(...)

In [552]:
juice_long = juice_df.melt(id_vars=['title', 'year'], value_vars=['total_words', 'unique_words'],
                           var_name='word_type', value_name='count')

scatter_plot = alt.Chart(juice_long).mark_circle(size=100).encode(
    x=alt.X('year:O', title='Year'),
    y=alt.Y('count:Q', title='Word Count'),
    color=alt.Color('word_type:N', title='Word Type',
                    scale=alt.Scale(domain=['total_words', 'unique_words'], range=['#1f77b4', '#ff7f0e'])),
    tooltip=['title:N', 'year:O', 'word_type:N', 'count:Q']
).properties(
    width=600,
    height=400,
    title='Total Words vs Unique Words Per Year'
)

scatter_plot

alt.Chart(...)

# Using UMAP model and SentenceTransformer to extract topics from lyrics  

After preprocessing the text by removing headers, punctiations, numbers and stopwords, using different embedding techniques, the results produced were not ideal and do not provide useful information. This may be do to the nature of the text and could be improved with deeper processing strategies. I also tried to use the BERT visualization but after trying different solutions, it still did not work. 

In [476]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
topic_model = BERTopic(embedding_model=embedding_model)
topics, probs = topic_model.fit_transform(juice_df['cleaned_lyrics'].tolist())

In [477]:
umap_model = UMAP(n_components=5, random_state=29)
topic_model = BERTopic(umap_model=umap_model)
lyrics = juice_df['cleaned_lyrics'].tolist()

topics, probs = topic_model.fit_transform(lyrics)

juice_df['topic'] = topics

In [482]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,39,-1_im_yeah_know_dont,"[im, yeah, know, dont, like, got, oh, love, fe...",[im favorite place feel place still runnin opt...
1,0,24,0_yeah_uh_im_like,"[yeah, uh, im, like, bitch, got, dont, fuck, u...",[contributorstranslationsукраїнськарусскийespa...
2,1,21,1_love_im_oh_tell,"[love, im, oh, tell, dont, yeah, know, one, go...",[woah uh oh ohoh oh mello made right uh try ev...


In [466]:
juice_df

,title,lyrics,release_date_components,year,topics,topic,cleaned_lyrics
0,Lucid Dreams,"Enviyon on the mix\nNo, no, no, no\nNo-no, no,...","{'year': 2017, 'month': 6, 'day': 15}",2017.0,2,1,enviyon mix nono still see shadows room cant t...
1,All Girls Are the Same,"Mhm, they're rotting my brain, love\nThese hoe...","{'year': 2017, 'month': 12, 'day': 22}",2017.0,-1,1,mhm theyre rotting brain love hoes admit anoth...
2,Wishing Well,"Mm-mm, mm-mm-mm-mm, mm-mm-mm, uh\nI can't brea...","{'year': 2020, 'month': 7, 'day': 10}",2020.0,0,-1,mmmm mmmmmmmm mmmmmm uh cant breathe chopsquad...
3,Robbery,"Man, what?\nThis shit funny, one second\nOof, ...","{'year': 2019, 'month': 2, 'day': 13}",2019.0,-1,-1,man shit funny one second oof hennessy strong ...
4,Legends,"I usually don't speak on shit like this, but\n...","{'year': 2018, 'month': 6, 'day': 19}",2018.0,0,1,usually dont speak shit like aint nobody else ...
...,...,...,...,...,...,...,...
79,Ronald,"Uh, yeah, huh\n\n\nWhat's that belt? Hermès, u...","{'year': 2019, 'month': 3, 'day': None}",2019.0,-1,0,uh yeah huh whats belt hermès uh uhhuh whats n...
80,Arctic Tundra [Original],"808 Mafia\nHaha, Max Lord, preach it, haha\nGe...",None,NaN,-1,0,mafia haha max lord preach haha gezin fuck blu...
81,Adore You,"Let me talk to y'all for a second, though\nLet...","{'year': 2024, 'month': 11, 'day': 29}",2024.0,-1,-1,let talk yall second though let get lights let...
82,Confide,"Uh, I need to confide\nI need to get right\nIt...",None,NaN,2,1,uh need confide need get right long ride uh dr...


In [448]:
topic_model.visualize_topics()

C:\Users\Sofia\Documents\Anaconda\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(


TypeError: Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.